In [1]:
import sys
sys.path.append("../../")
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

In [2]:
from cellmating.tracking._hausdorf_tracker import Tracker
from cellmating.visualize import animate_images
from cellmating.utils import file_traverse
from cellmating.io import imread, imsave
from cellmating.utils import crop_single_object

In [3]:
file_list = file_traverse("/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/", file_regular=".*._rigid.tif$")

In [4]:
filename = file_list[1]
print(filename)
image = imread(filename)
image = np.moveaxis(image, -1, 1)
print(image.shape)

/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/YAV2104xYAV2105_005_02_rigid.tif
(241, 4, 1020, 1020)


In [5]:
trace = Tracker(image[:, -1], max_miss=10, min_hist=3, low_ratio=0.6, high_ratio=1.5)
_ = trace()
traced_image = trace.to_image()

frame= 1 [58] [0, 12, 21, 80, 90, 100]
frame= 2 [58] [0, 12, 21]
frame= 3 [58] [0, 12, 21, 98, 101]
frame= 4 [58] [0, 12]
frame= 5 [58] [0, 20, 97, 100]
divison: [[1 4 5]]
frame= 6 [58] [0, 12, 21, 62]
divison: [[0 3 4]]
frame= 7 [58] [0, 12, 21, 41, 63]
frame= 8 [58] [0, 12, 21, 62]
frame= 9 [58] [0, 12, 40, 63, 81, 101]
frame= 10 [58] [0, 12, 21, 100]
divison: [[0 4 6]]
frame= 11 [] [0, 12, 21, 41, 64, 84]
frame= 12 [] [0, 39, 83]
divison: [[0 2 3]]
frame= 13 [] [0, 8, 41, 85]
divison: [[0 1 2]]
frame= 14 [] [0]
frame= 15 [] [0, 8, 64, 86]
divison: [[0 1 2]
 [1 4 5]]
frame= 16 [] [0, 41]
divison: [[0 2 3]]
frame= 17 [] [0, 12, 55]
frame= 18 [] [0, 12]
frame= 19 [] [0, 12, 21, 106, 110]
frame= 20 [] [0, 8, 13, 22, 45, 87]
frame= 21 [] [0, 8, 13, 22, 45]
frame= 22 [] [0, 12, 21, 44, 87]
divison: [[0 8 9]]
frame= 23 [] [0, 3, 9, 14, 23, 46, 70, 71, 112]
divison: [[0 5 6]]
frame= 24 [] [0, 3, 13, 22, 45, 90]
divison: [[0 2 3]]
frame= 25 [] [0, 3, 14, 23, 46]
frame= 26 [] [0, 3, 14, 46, 1

In [19]:
annotation = label2rgb(traced_image)

ani = animate_images(image[:, -1,:,:])
ani2 = animate_images(annotation[:])

html1 = HTML(ani.to_jshtml()).data
html2 = HTML(ani2.to_jshtml()).data

# Display the combined animations
HTML(f"""
<div style="display: flex;">
    <div style="flex: 1; margin-right: 10px;">
        {html1}
    </div>
    <div style="flex: 1; margin-left: 10px;">
        {html2}
    </div>
</div>
""")

NameError: name 'label2rgb' is not defined

In [ ]:
traced_image_stack = np.concatenate((image, traced_image[:,None,:,:]), axis=1)
imsave(filename[:-4]+"_track.tif", traced_image_stack, dtype=np.uint16, imagej=True)

In [6]:
import yaml

In [ ]:
import yaml
f = open('meta.yaml', 'w+')
yaml.dump(output_data, f, allow_unicode=True)